# Yuri Project

# GPT2 기반

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


In [ ]:
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
prompt = "어떻게 사람이름이 엄준식이지?"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=1.0,
    max_length=500,
)
gen_text = tokenizer.batch_decode(gen_tokens)[0]
print(gen_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


어떻게 사람이름이 엄준식이지? 교라자 교마태를하런 되���이 매썍어스 매쌬습잠 앆면지 위니이 지지?

튬맑아 사람다로 습니감 진을 전관떻게 알어게을 자었도 예면스 [랓면하 맹시?]듾이 욘식기립……

교라자 교마태를하런 되���이 매썍어스 교라자 었도 예면스 아드 다 것땼을 맄라가 팘드 않리를자라잀 라자 교라잀? 지나수 예라을 있우가 팘드 면니다는 전관떴게 팘드 않리를자라잀라잀라자가 찈야굌다 그넙이 에지?

애�


# GPT-2 Fine-tuning

In [ ]:
!pip install accelerate -U
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import GPT2Tokenizer, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
!pip install datasets
from datasets import load_dataset, load_metric, DatasetDict
import urllib.request
import pandas as pd

!git clone https://github.com/smilegate-ai/HuLiC.git

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Data collator 인스턴스 생성
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenizer.pad_token = tokenizer.eos_token

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
df = pd.read_excel('/content/HuLiC/HuLiC_benchmark/일상대화_Human.xlsx',sheet_name = 0)

In [ ]:
print(df.head())

In [ ]:
class ConversationDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.examples = []

        for _, row in dataframe.iterrows():
            concatenated_text = f"{row['Question']} {tokenizer.eos_token} {row['Answer']}"
            # 확실하게 텐서 반환을 보장
            inputs = tokenizer(concatenated_text, truncation=True, max_length=512, padding="max_length", return_tensors='pt')
            # labels를 생성하기 위해 input_ids의 텐서 복사본을 사용
            self.examples.append({
                'input_ids': inputs['input_ids'].squeeze(0),
                'attention_mask': inputs['attention_mask'].squeeze(0),
                'labels': inputs['input_ids'].squeeze(0).clone()
            })

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]


In [ ]:
dataset = ConversationDataset(df, tokenizer)

In [ ]:
data_loader = DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    warmup_steps=100,
    weight_decay=0.03,
    logging_dir='./logs',
    logging_steps=50,
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator  # 데이터 콜레이터 추가
)


In [ ]:
trainer.train()
model.save_pretrained('./fine_tuned_model')

In [ ]:
# 테스트


## 다른버전

In [ ]:
ls checkpoint/fine_tuning_run_1

encoder.json  hparams.json  vocab.bpe


In [ ]:
file_name = pd.read_csv('OPELA/data/oplea_open_data.csv')
run_name = 'fine_tuning_run_1'
model_size = '355M'
ops.reset_default_graph()

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")

('ChatBotData.csv', <http.client.HTTPMessage at 0x7f0d938229b0>)

In [ ]:
file_name = pd.read_csv('ChatBotData.csv')
run_name = 'fine_tuning_run_1'
model_size = '355M'
ops.reset_default_graph()

In [ ]:
df = pd.read_csv('ChatBotData.csv')
print(df.head())  # Print the first few rows of the DataFrame
print(df.describe())  # Get a statistical summary of the DataFrame
print(df.isnull().sum())  # Check for missing values in each column

                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0
              label
count  11823.000000
mean       0.803180
std        0.812012
min        0.000000
25%        0.000000
50%        1.000000
75%        2.000000
max        2.000000
Q        0
A        0
label    0
dtype: int64


## 파인튜닝

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_size,
              steps=2000,
              restore_from='fresh',
              run_name = run_name,
              print_every=10,
              sample_every=50,
              save_every=50)
             # , learning_rate=.00003)

For larger models, the recommended finetune() parameters are:
	use_memory_saving_gradients = True
	only_train_transformer_layers = True
	accumulate_gradients = 1

Loading checkpoint models/355M/model.ckpt
Loading dataset...


TypeError: stat: path should be string, bytes, os.PathLike or integer, not DataFrame

In [ ]:
input_prompt = "날씨가 좋네"
gpt2.generate(sess, run_name=run_name, temperature=0.5, length=50, prefix=input_prompt, top_k=200, nsamples=10)

���씨가 좋네이 선물도 소녀를 다리리 아무서 사람들어 �
���씨가 좋네예을 못해서 바란 내재를 않았세요. 있었어
���씨가 좋네안는 것이서리를 이제 있어요. 우리하면 �
���씨가 좋네에 날씨가 방야을 번제 가아요.


보어를 �
���씨가 좋네는 하면 아니다.

지난 이어요 = I'm going to the park.

지난
���씨가 좋네에 대하지 시작들에 있는 다시 있었습니다. �
���씨가 좋네를 없어요 = I want to take your car and drive you to my house

정제지 서우 중어�
���씨가 좋네이 선상하고.

정말러 방송도 아나는 공지
���씨가 좋네리게 날씨가 좋네리게 날씨가 좋네리게 �
���씨가 좋네이 이나이 못장이 많이 고무 모들니다. 그런 �


# LLAMA 기반

In [ ]:
!pip install torch
!pip install transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
import transformers
import torch

In [ ]:
model_id = "meta-llama/Meta-Llama-3-70B"

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-70B.
401 Client Error. (Request ID: Root=1-6645feaf-7c3491013b1287af07f5b6c9;067d8899-67a2-49d1-8b88-c3d7293ac6e0)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-70B/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-70B is restricted. You must be authenticated to access it.

In [ ]:
pipeline("Hey how are you doing today?")